# VacationPy

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [60]:
path = "../output_data/cities.csv"
cities = pd.read_csv(path)
cities.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,Sept-Îles,CA,50.2001,-66.3821,23.00,58,15,8.05,1611788931
1,Nyurba,RU,63.2842,118.3319,-47.02,78,95,1.45,1611789246
2,Haines Junction,CA,60.7522,-137.5108,-9.40,54,83,0.20,1611788875
3,Hobart,AU,-42.8794,147.3294,59.00,47,75,8.05,1611788991
4,Butaritari,KI,3.0707,172.7902,80.20,81,82,13.24,1611789085


# Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight
- Add Heatmap layer to map.

In [65]:
gmaps.configure(api_key=g_key)
locations = cities[['Latitude', 'Longitude']]

humidity = cities["Humidity (%)"]

fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities["Humidity (%)"]),
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [66]:
vacation_df= cities.loc[(cities['Max Temperature (F)']<=85) & (cities['Max Temperature (F)']>=70) 
                        & (cities['Wind Speed (mph)']<=5) 
                        & (cities['Cloudiness (%)']<=10)]

vacation_df = vacation_df.dropna()
vacation_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
16,Cap Malheureux,MU,-19.9842,57.6142,77.00,81,2,1.99,1611788933
180,Saint-Louis,SN,16.3333,-15.0000,73.40,28,0,2.30,1611788955
252,Morondava,MG,-20.2833,44.2833,82.22,74,0,2.84,1611789279
295,Nioro,GM,13.3500,-15.7500,73.76,17,0,1.45,1611788912
332,Dawei,MM,14.0833,98.2000,72.18,80,0,3.74,1611788977
367,Vila Velha,BR,-20.3297,-40.2925,78.80,73,0,2.30,1611788914
403,Koungou,YT,-12.7336,45.2042,78.80,83,0,3.44,1611788987
420,São Francisco,BR,-15.9486,-44.8644,72.68,61,0,4.05,1611788990
424,Les Cayes,HT,18.2000,-73.7500,75.69,70,0,4.00,1611788990
454,Maceió,BR,-9.6658,-35.7353,75.20,88,0,3.44,1611789303


# Hotel Map

- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [67]:
hotel_df = vacation_df
hotel_df["Hotel Name"] = ""


target_radius = 5000
target_type = "lodging"
gkey = g_key
keyword = "hotel"

# Set params
params = {"radius": target_radius,
    "type": target_type,
    "key": gkey,
      "keyword" : keyword}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


# hotel_df = vacation_df['Hotel Name'].tolist()

Missing field/result... skipping.
Missing field/result... skipping.


In [68]:
# Visualize to confirm hotel name appears
hotel_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name
16,Cap Malheureux,MU,-19.9842,57.6142,77.00,81,2,1.99,1611788933,Paradise Cove Boutique Hotel
180,Saint-Louis,SN,16.3333,-15.0000,73.40,28,0,2.30,1611788955,
252,Morondava,MG,-20.2833,44.2833,82.22,74,0,2.84,1611789279,Palissandre Cote Ouest resort & SPA
295,Nioro,GM,13.3500,-15.7500,73.76,17,0,1.45,1611788912,
332,Dawei,MM,14.0833,98.2000,72.18,80,0,3.74,1611788977,The Mandolis Hotel and Restaurant
367,Vila Velha,BR,-20.3297,-40.2925,78.80,73,0,2.30,1611788914,Golden Tulip Porto Vitória
403,Koungou,YT,-12.7336,45.2042,78.80,83,0,3.44,1611788987,LHACIENDA976
420,São Francisco,BR,-15.9486,-44.8644,72.68,61,0,4.05,1611788990,Hotel Supremo
424,Les Cayes,HT,18.2000,-73.7500,75.69,70,0,4.00,1611788990,Regi Hotel & Bar
454,Maceió,BR,-9.6658,-35.7353,75.20,88,0,3.44,1611789303,Jatiúca Hotel & Resort


In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [70]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig = gmaps.figure(map_type="HYBRID")


markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))